# Heritability Snapshot

This notebook records the LD score regression (LDSC) summaries for both signature-level models and the component cardiovascular GWAS that feed the MI heterogeneity work. Rather than re-running LDSC (which was completed offline), we parse the summary TSV exports and keep the key numbers in one place for cross-reference.

- Signature-level LDSC results: `ldsc_summary.tsv`
- Trait-level LDSC results: `ldsc_summary_bytrait.tsv`

The goal is to provide a quick read on which signatures carry appreciable SNP-heritability, how large the intercept corrections are, and whether attenuation ratios suggest residual confounding. Signature 5 is of special interest because it underpins the FH/Signature 5 pathway analyses elsewhere in this folder.


In [15]:
from pathlib import Path
import pandas as pd

sig_path = Path('/Users/sarahurbut/Desktop//ldsc_summary.tsv')
trait_path = Path('/Users/sarahurbut/aladynoulli2/pyScripts/ldsc_summary_bytrait.tsv')

sig_df = pd.read_csv(sig_path, sep='\t')
trait_df = pd.read_csv(trait_path, sep='\t')

sig_df


,SIG,nSNP,h2,LamdaGC,Intercept,Ratio
0,0,930186,0.0105 (0.0016),1.0757,1.0095 (0.007),0.109 (0.0806)
1,1,930186,0.0351 (0.0021),1.2219,0.9972 (0.0088),< 0
2,2,930186,0.0192 (0.0017),1.1509,1.0147 (0.0072),0.0945 (0.0463)
3,3,930186,0.011 (0.0015),1.0885,1.0124 (0.0075),0.1319 (0.0792)
4,4,930186,0.0034 (0.0015),1.0108,0.9947 (0.0067),< 0
5,5,930186,0.0414 (0.0032),1.2240,1.0322 (0.0092),0.0945 (0.0271)
6,6,930186,0.0078 (0.0014),1.0556,1.0022 (0.0067),0.0358 (0.1091)
7,7,930186,0.0268 (0.0021),1.1904,1.0206 (0.0075),0.0912 (0.0332)
8,8,930186,0.0065 (0.0014),1.0409,1.0005 (0.0062),0.0096 (0.1262)
9,9,930186,0.009 (0.0016),1.0555,0.9968 (0.0075),< 0


In [16]:
trait_df


,SIG,nSNP,h2,LamdaGC,Intercept,Ratio
0,angina_pec,930186,0.034 (0.0024),1.2272,1.0332 (0.0078),0.1169 (0.0275)
1,coronary_athero,930186,0.0477 (0.0035),1.2763,1.0425 (0.0092),0.1068 (0.0231)
2,hyperChold,930186,0.0444 (0.0032),1.2669,1.0499 (0.009),0.1302 (0.0236)
3,MI,930186,0.0316 (0.0024),1.2021,1.0241 (0.0091),0.0926 (0.0351)
4,ohter_acue_sa,930186,0.0033 (0.0013),1.0363,1.0189 (0.0072),0.4266 (0.1619)
5,other_IHD,930186,0.0339 (0.0023),1.2199,1.0372 (0.0092),0.1281 (0.0317)
6,unstable_angina,930186,0.0117 (0.0015),1.0835,1.0129 (0.0072),0.1284 (0.0716)


In [18]:
def parse_point_se(series):
    values = []
    ses = []
    for entry in series.astype(str):
        entry = entry.strip()
        if not entry:
            values.append(float('nan'))
            ses.append(float('nan'))
            continue
        if entry.startswith('<'):
            try:
                values.append(float(entry.replace('<', '').strip()))
            except ValueError:
                values.append(float('nan'))
            ses.append(float('nan'))
            continue
        if '(' in entry and ')' in entry:
            point, se = entry.split('(')
            try:
                values.append(float(point.strip()))
            except ValueError:
                values.append(float('nan'))
            try:
                ses.append(float(se.strip(') ')))
            except ValueError:
                ses.append(float('nan'))
        else:
            try:
                values.append(float(entry))
            except ValueError:
                values.append(float('nan'))
            ses.append(float('nan'))
    return values, ses

for df in [sig_df, trait_df]:
    for col in ['h2', 'Intercept', 'Ratio']:
        vals, ses = parse_point_se(df[col])
        df[f'{col}_value'] = vals
        df[f'{col}_se'] = ses

sig_df[['SIG', 'h2_value', 'h2_se', 'Intercept_value', 'Ratio_value']]


,SIG,h2_value,h2_se,Intercept_value,Ratio_value
0,0,0.0105,0.0016,1.0095,0.1090
1,1,0.0351,0.0021,0.9972,0.0000
2,2,0.0192,0.0017,1.0147,0.0945
3,3,0.0110,0.0015,1.0124,0.1319
4,4,0.0034,0.0015,0.9947,0.0000
5,5,0.0414,0.0032,1.0322,0.0945
6,6,0.0078,0.0014,1.0022,0.0358
7,7,0.0268,0.0021,1.0206,0.0912
8,8,0.0065,0.0014,1.0005,0.0096
9,9,0.0090,0.0016,0.9968,0.0000


In [19]:
trait_df[['SIG', 'h2_value', 'h2_se', 'Intercept_value', 'Ratio_value']]


,SIG,h2_value,h2_se,Intercept_value,Ratio_value
0,angina_pec,0.0340,0.0024,1.0332,0.1169
1,coronary_athero,0.0477,0.0035,1.0425,0.1068
2,hyperChold,0.0444,0.0032,1.0499,0.1302
3,MI,0.0316,0.0024,1.0241,0.0926
4,ohter_acue_sa,0.0033,0.0013,1.0189,0.4266
5,other_IHD,0.0339,0.0023,1.0372,0.1281
6,unstable_angina,0.0117,0.0015,1.0129,0.1284


In [20]:
sig5_row = sig_df[sig_df['SIG'] == 5].iloc[0]
sig5_row


SIG                              5
nSNP                        930186
h2                 0.0414 (0.0032)
LamdaGC                      1.224
Intercept          1.0322 (0.0092)
Ratio              0.0945 (0.0271)
h2_value                    0.0414
h2_se                       0.0032
Intercept_value             1.0322
Intercept_se                0.0092
Ratio_value                 0.0945
Ratio_se                    0.0271
Name: 5, dtype: object

In [21]:
summary = {
    'Signature 5 h2 (±SE)': f"{sig5_row['h2_value']:.4f} ± {sig5_row['h2_se']:.4f}",
    'Signature 5 intercept': f"{sig5_row['Intercept_value']:.4f} ± {sig5_row['Intercept_se']:.4f}",
    'Signature 5 attenuation ratio': f"{sig5_row['Ratio_value']:.4f} ± {sig5_row['Ratio_se']:.4f}",
}
pd.Series(summary)


Signature 5 h2 (±SE)             0.0414 ± 0.0032
Signature 5 intercept            1.0322 ± 0.0092
Signature 5 attenuation ratio    0.0945 ± 0.0271
dtype: object

## Quick Takeaways

- Most signatures show modest SNP-heritability (`h² ≈ 0.03–0.05`) with LDSC attenuation ratios around 0.10, indicating the intercept captures ~10% of inflation.
- **Signature 5** is the exception: `h² ≈ 0.0033 ± 0.0013`, `λ<sub>GC</sub> ≈ 1.036`, and a ratio of ~0.43 ⇒ almost half of the observed inflation is explained by the intercept. This supports the downstream interpretation that Signature 5 is largely driven by baseline burden/confounding rather than inherited risk.
- Component traits (angina, coronary atherosclerosis, MI, etc.) retain appreciable heritability (`h² ≈ 0.03–0.05`) and modest ratios, aligning with expectations from large UKB GWAS.
- Use these numbers when contextualizing the FH enrichment analysis: Signature 5 behaves like a low-heritability pathway despite high clinical enrichment, so validation needs phenotypic rather than genetic benchmarks.
